<a href="https://colab.research.google.com/github/lucasmark07/SummerCapstone25/blob/main/Dynamic%20Parking%20Pricing%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 300.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 

### *main.py*
(includes pricing models, batch processing and Pathway integration.)

In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
import pathway as pw

# Define the pricing functions directly
def model_1_pricing(current_price, occupancy, capacity, alpha):
    """Calculates the next price using the Baseline Linear Model."""
    return current_price + alpha * (occupancy / capacity)

def model_2_pricing(base_price, occupancy, capacity, queue_length, traffic_condition, is_special_day, vehicle_type, lambda_val, alpha, beta, gamma, delta, epsilon):
    """Calculates the price using the Demand-Based Price Function."""
    # Feature Engineering for VehicleType
    vehicle_type_weights = {
        'car': 1.0,
        'bike': 0.5,
        'truck': 1.5,
        'cycle': 0.3
    }
    vehicle_type_weight = vehicle_type_weights.get(vehicle_type, 1.0) # Default to 1.0 if unknown

    # Feature Engineering for TrafficConditionNearby
    traffic_weights = {
        'low': 0.5,
        'average': 1.0,
        'high': 1.5
    }
    traffic_level = traffic_weights.get(traffic_condition, 1.0) # Default to 1.0 if unknown

    # Calculate demand
    demand = alpha * (occupancy / capacity) + beta * queue_length - gamma * traffic_level + delta * is_special_day + epsilon * vehicle_type_weight

    # Normalize demand (simple min-max normalization for demonstration, needs proper range based on data analysis)
    # Assuming a hypothetical min_demand and max_demand for normalization. These should be derived from data.
    min_demand = -5.0
    max_demand = 15.0
    normalized_demand = (demand - min_demand) / (max_demand - min_demand)
    normalized_demand = np.clip(normalized_demand, 0, 1) # Ensure it's between 0 and 1

    # Calculate price
    price = base_price * (1 + lambda_val * normalized_demand)

    # Apply price bounds
    min_price = base_price * 0.5
    max_price = base_price * 2.0
    return np.clip(price, min_price, max_price)

def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points on the earth (specified in decimal degrees)"""
    R = 6371  # Radius of Earth in kilometers

    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def model_3_pricing(current_price_model2, occupancy, capacity, lat, lon, current_system_code, all_parking_data_at_timestep, base_price):
    """Calculates the price using the Competitive Pricing Model."""
    competitive_factor = 0.0
    nearby_competitors = []

    for idx, competitor_row in all_parking_data_at_timestep.iterrows():
        if competitor_row["SystemCodeNumber"] != current_system_code:
            distance = haversine(lat, lon, competitor_row["Latitude"], competitor_row["Longitude"])
            if distance < 1.0: # Consider competitors within 1 km radius
                nearby_competitors.append(competitor_row)

    if nearby_competitors:
        avg_competitor_price = np.mean([comp["Price_Model2"] for comp in nearby_competitors])

        occupancy_rate = occupancy / capacity
        if occupancy_rate > 0.8 and avg_competitor_price < current_price_model2:
            competitive_factor = -0.1 # Reduce price by 10%
        elif avg_competitor_price > current_price_model2 * 1.2: # If competitors are significantly more expensive
            competitive_factor = 0.05 # Increase price by 5%

    return current_price_model2 * (1 + competitive_factor)

# Define the schema for the streaming data (Pathway)
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Latitude: float
    Longitude: float
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: bool
    VehicleType: str

def batch_processing(dataset_path, output_path):
    """Process the dataset in batch mode using pandas"""
    print("Starting batch processing...")

    # Load the dataset
    df = pd.read_csv(dataset_path)

    # Convert 'LastUpdatedDate' and 'LastUpdatedTime' to datetime objects
    df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")

    # Sort by SystemCodeNumber and Timestamp to ensure correct chronological processing per parking lot
    df = df.sort_values(by=["SystemCodeNumber", "Timestamp"]).reset_index(drop=True)

    # Initialize 'Price' columns with the base price of $10
    df["Price_Model1"] = 10.0
    df["Price_Model2"] = 10.0
    df["Price_Model3"] = 10.0

    # --- Model 1 Implementation ---
    alpha_model1 = 0.05 # Example alpha value for Model 1

    # Process data for each parking lot independently for Model 1
    for system_code in df["SystemCodeNumber"].unique():
        lot_data_indices = df[df["SystemCodeNumber"] == system_code].index
        current_price = 10.0 # Starting price for each lot
        for i, original_idx in enumerate(lot_data_indices):
            if i > 0:
                current_price = df.loc[lot_data_indices[i-1], "Price_Model1"]

            new_price = model_1_pricing(current_price, df.loc[original_idx, "Occupancy"], df.loc[original_idx, "Capacity"], alpha_model1)
            df.loc[original_idx, "Price_Model1"] = new_price

    # --- Model 2 Implementation ---
    alpha_model2 = 0.1
    beta_model2 = 0.2
    gamma_model2 = 0.1
    delta_model2 = 0.5
    epsilon_model2 = 0.3
    lambda_model2 = 0.5

    # Process data for each parking lot independently for Model 2
    for system_code in df["SystemCodeNumber"].unique():
        lot_data_indices = df[df["SystemCodeNumber"] == system_code].index
        base_price = 10.0 # Base price for each lot
        for i, original_idx in enumerate(lot_data_indices):
            new_price = model_2_pricing(
                base_price,
                df.loc[original_idx, "Occupancy"],
                df.loc[original_idx, "Capacity"],
                df.loc[original_idx, "QueueLength"],
                df.loc[original_idx, "TrafficConditionNearby"],
                df.loc[original_idx, "IsSpecialDay"],
                df.loc[original_idx, "VehicleType"],
                lambda_model2,
                alpha_model2,
                beta_model2,
                gamma_model2,
                delta_model2,
                epsilon_model2
            )
            df.loc[original_idx, "Price_Model2"] = new_price

    # --- Model 3 Implementation ---
    # This model requires iterating through time steps to get competitor prices at each step
    # We will group by Timestamp to process all parking lots at a given time step together

    df["Price_Model3"] = df["Price_Model2"] # Start Model 3 price with Model 2 price

    for timestamp in df["Timestamp"].unique():
        # Get all parking data for the current timestamp
        current_timestep_data = df[df["Timestamp"] == timestamp].copy()

        for original_idx in current_timestep_data.index:
            system_code = df.loc[original_idx, "SystemCodeNumber"]
            occupancy = df.loc[original_idx, "Occupancy"]
            capacity = df.loc[original_idx, "Capacity"]
            lat = df.loc[original_idx, "Latitude"]
            lon = df.loc[original_idx, "Longitude"]
            base_price = 10.0 # Base price for Model 3 is also $10
            current_price_model2 = df.loc[original_idx, "Price_Model2"]

            new_price_model3 = model_3_pricing(
                current_price_model2,
                occupancy,
                capacity,
                lat,
                lon,
                system_code,
                current_timestep_data, # Pass data for all lots at this timestamp
                base_price
            )
            df.loc[original_idx, "Price_Model3"] = new_price_model3

    # Save the processed data with new prices
    df.to_csv(output_path, index=False)
    print(f"Batch processing completed. Output saved to {output_path}")
    return df

def streaming_processing(dataset_path, stream_output_path):
    """Process the dataset in streaming mode using Pathway"""
    print("Starting streaming processing with Pathway...")

    try:
        # Read the original dataset to extract relevant columns for Pathway
        df_original = pd.read_csv(dataset_path)

        # Select a subset of columns that are relevant for the Pathway stream
        stream_df = df_original[[
            'SystemCodeNumber', 'Capacity', 'Occupancy', 'LastUpdatedDate',
            'LastUpdatedTime', 'Latitude', 'Longitude', 'QueueLength',
            'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType'
        ]].copy()

        # Save this subset to a temporary CSV file for Pathway to read
        stream_file_path = '/tmp/parking_stream_for_pathway.csv'
        stream_df.to_csv(stream_file_path, index=False)

        # Load the data as a simulated stream using Pathway's replay_csv function
        data = pw.demo.replay_csv(stream_file_path, schema=ParkingSchema, input_rate=1000)

        # Add a Timestamp column to the Pathway data stream
        fmt = "%d-%m-%Y %H:%M:%S"
        data_with_time = data.with_columns(
            Timestamp=pw.this.LastUpdatedDate + " " + pw.this.LastUpdatedTime
        ).with_columns(
            t=pw.this.Timestamp.dt.strptime(fmt)
        )

        # --- Pathway Integration for Pricing Models ---

        alpha_model1_pw = 0.05

        # Model 1 in Pathway
        model1_output = data_with_time.with_columns(
            Price_Model1 = pw.apply(
                lambda occupancy, capacity: model_1_pricing(10.0, occupancy, capacity, alpha_model1_pw),
                pw.this.Occupancy, pw.this.Capacity
            )
        )

        # Model 2 in Pathway
        alpha_model2_pw = 0.1
        beta_model2_pw = 0.2
        gamma_model2_pw = 0.1
        delta_model2_pw = 0.5
        epsilon_model2_pw = 0.3
        lambda_model2_pw = 0.5

        model2_output = model1_output.with_columns(
            Price_Model2 = pw.apply(
                lambda occupancy, capacity, queue_length, traffic_condition, is_special_day, vehicle_type: model_2_pricing(
                    10.0, # Base price
                    occupancy,
                    capacity,
                    queue_length,
                    traffic_condition,
                    is_special_day,
                    vehicle_type,
                    lambda_model2_pw,
                    alpha_model2_pw,
                    beta_model2_pw,
                    gamma_model2_pw,
                    delta_model2_pw,
                    epsilon_model2_pw
                ),
                pw.this.Occupancy,
                pw.this.Capacity,
                pw.this.QueueLength,
                pw.this.TrafficConditionNearby,
                pw.this.IsSpecialDay,
                pw.this.VehicleType
            )
        )

        # Model 3 in Pathway - Simplified version
        # Note: This is a simplified implementation since Model 3 requires competitor data
        # In a real streaming scenario, you'd need proper joins with competitor data
        model3_output = model2_output.with_columns(
            Price_Model3 = pw.apply(
                lambda price_model2, occupancy, capacity: price_model2 * (1 + (0.05 if occupancy/capacity > 0.8 else 0)),
                pw.this.Price_Model2,
                pw.this.Occupancy,
                pw.this.Capacity
            )
        )

        # Output the results to a CSV file (for demonstration)
        pw.io.csv.write(model3_output, stream_output_path)
        print(f"Streaming processing setup completed. Output will be saved to {stream_output_path}")

        return model3_output

    except ImportError:
        print("Pathway library not available. Skipping streaming processing.")
        return None
    except Exception as e:
        print(f"Error in streaming processing: {e}")
        return None

def main():
    """Main function to run both batch and streaming processing"""
    dataset_path = "/mnt/upload/dataset.csv"
    batch_output_path = "/mnt/processed_dataset_batch.csv"
    stream_output_path = "/mnt/processed_dataset_stream.csv"

    print("=" * 60)
    print("PARKING PRICING SYSTEM - COMBINED PROCESSING")
    print("=" * 60)

    # Run batch processing
    print("\n1. BATCH PROCESSING")
    print("-" * 30)
    try:
        batch_df = batch_processing(dataset_path, batch_output_path)
        print(f"Batch processing successful! Processed {len(batch_df)} records.")

        # Display sample results
        print("\nSample batch processing results:")
        print(batch_df[['SystemCodeNumber', 'Occupancy', 'Capacity', 'Price_Model1', 'Price_Model2', 'Price_Model3']].head())

    except Exception as e:
        print(f"Batch processing failed: {e}")

    # Run streaming processing
    print("\n2. STREAMING PROCESSING")
    print("-" * 30)
    try:
        stream_result = streaming_processing(dataset_path, stream_output_path)
        if stream_result is not None:
            print("Streaming processing setup successful!")
        else:
            print("Streaming processing setup failed or skipped.")

    except Exception as e:
        print(f"Streaming processing failed: {e}")

    print("\n" + "=" * 60)
    print("PROCESSING COMPLETE")
    print("=" * 60)

if __name__ == "__main__":
    main()

PARKING PRICING SYSTEM - COMBINED PROCESSING

1. BATCH PROCESSING
------------------------------
Starting batch processing...
Batch processing completed. Output saved to /mnt/processed_dataset_batch.csv
Batch processing successful! Processed 18368 records.

Sample batch processing results:
  SystemCodeNumber  Occupancy  Capacity  Price_Model1  Price_Model2  \
0      BHMBCCMKT01         61       577     10.005286     11.365143   
1      BHMBCCMKT01         64       577     10.010832     11.365273   
2      BHMBCCMKT01         80       577     10.017764     11.415966   
3      BHMBCCMKT01        107       577     10.027036     11.417136   
4      BHMBCCMKT01        150       577     10.040035     11.381499   

   Price_Model3  
0     11.365143  
1     11.365273  
2     11.415966  
3     11.417136  
4     11.381499  

2. STREAMING PROCESSING
------------------------------
Starting streaming processing with Pathway...
Streaming processing setup completed. Output will be saved to /mnt/proce

### *visualisation.py*

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

# Set style for better-looking plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load the processed dataset
df = pd.read_csv("/mnt/processed_dataset_batch.csv")

# Convert Timestamp to datetime objects for plotting
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Add derived features for better analysis
df['Hour'] = df['Timestamp'].dt.hour
df['DayOfWeek'] = df['Timestamp'].dt.dayofweek
df['OccupancyRate'] = (df['Occupancy'] / df['Capacity']) * 100

# Select a few SystemCodeNumbers for plotting
sample_system_codes = df["SystemCodeNumber"].unique()[:3]

print(f"Analyzing data for {len(sample_system_codes)} system codes: {sample_system_codes}")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")

# 1. Enhanced Price Comparison Plots
for system_code in sample_system_codes:
    lot_data = df[df["SystemCodeNumber"] == system_code].sort_values(by="Timestamp")

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

    # Price trends
    ax1.plot(lot_data["Timestamp"], lot_data["Price_Model1"],
             label="Model 1", linewidth=2, alpha=0.8)
    ax1.plot(lot_data["Timestamp"], lot_data["Price_Model2"],
             label="Model 2", linewidth=2, alpha=0.8)
    ax1.plot(lot_data["Timestamp"], lot_data["Price_Model3"],
             label="Model 3", linewidth=2, alpha=0.8)

    ax1.set_ylabel("Price ($)", fontsize=12)
    ax1.set_title(f"Parking Price Comparison - System Code: {system_code}", fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)

    # Price difference analysis
    lot_data_copy = lot_data.copy()
    lot_data_copy['Model1_vs_Model2'] = lot_data_copy['Price_Model1'] - lot_data_copy['Price_Model2']
    lot_data_copy['Model1_vs_Model3'] = lot_data_copy['Price_Model1'] - lot_data_copy['Price_Model3']
    lot_data_copy['Model2_vs_Model3'] = lot_data_copy['Price_Model2'] - lot_data_copy['Price_Model3']

    ax2.plot(lot_data_copy["Timestamp"], lot_data_copy["Model1_vs_Model2"],
             label="Model 1 - Model 2", alpha=0.7)
    ax2.plot(lot_data_copy["Timestamp"], lot_data_copy["Model1_vs_Model3"],
             label="Model 1 - Model 3", alpha=0.7)
    ax2.plot(lot_data_copy["Timestamp"], lot_data_copy["Model2_vs_Model3"],
             label="Model 2 - Model 3", alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

    ax2.set_xlabel("Time", fontsize=12)
    ax2.set_ylabel("Price Difference ($)", fontsize=12)
    ax2.set_title("Price Model Differences", fontsize=12)
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"enhanced_price_comparison_{system_code}.png", dpi=300, bbox_inches='tight')
    plt.close()

# 2. Enhanced Occupancy Analysis
for system_code in sample_system_codes:
    lot_data = df[df["SystemCodeNumber"] == system_code].sort_values(by="Timestamp")

    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Occupancy vs Capacity
    axes[0,0].plot(lot_data["Timestamp"], lot_data["Occupancy"],
                   label="Occupancy", color="steelblue", linewidth=2)
    axes[0,0].axhline(y=lot_data["Capacity"].iloc[0], color="red",
                      linestyle="--", label="Capacity", linewidth=2)
    axes[0,0].fill_between(lot_data["Timestamp"], lot_data["Occupancy"],
                           alpha=0.3, color="steelblue")
    axes[0,0].set_ylabel("Count", fontsize=12)
    axes[0,0].set_title(f"Occupancy vs Capacity - System {system_code}", fontsize=12, fontweight='bold')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # Occupancy Rate
    axes[0,1].plot(lot_data["Timestamp"], lot_data["OccupancyRate"],
                   color="orange", linewidth=2)
    axes[0,1].axhline(y=100, color="red", linestyle="--", alpha=0.7, label="100% Capacity")
    axes[0,1].axhline(y=80, color="yellow", linestyle="--", alpha=0.7, label="80% Capacity")
    axes[0,1].set_ylabel("Occupancy Rate (%)", fontsize=12)
    axes[0,1].set_title("Occupancy Rate Over Time", fontsize=12)
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)

    # Queue Length
    axes[1,0].plot(lot_data["Timestamp"], lot_data["QueueLength"],
                   color="purple", linewidth=2)
    axes[1,0].fill_between(lot_data["Timestamp"], lot_data["QueueLength"],
                           alpha=0.3, color="purple")
    axes[1,0].set_xlabel("Time", fontsize=12)
    axes[1,0].set_ylabel("Queue Length", fontsize=12)
    axes[1,0].set_title("Queue Length Trend", fontsize=12)
    axes[1,0].grid(True, alpha=0.3)

    # Hourly patterns
    hourly_avg = lot_data.groupby('Hour').agg({
        'OccupancyRate': 'mean',
        'QueueLength': 'mean'
    }).reset_index()

    ax_twin = axes[1,1]
    ax_twin2 = ax_twin.twinx()

    line1 = ax_twin.bar(hourly_avg['Hour'], hourly_avg['OccupancyRate'],
                        alpha=0.7, color='steelblue', label='Avg Occupancy Rate')
    line2 = ax_twin2.plot(hourly_avg['Hour'], hourly_avg['QueueLength'],
                          color='red', marker='o', linewidth=2, label='Avg Queue Length')

    ax_twin.set_xlabel("Hour of Day", fontsize=12)
    ax_twin.set_ylabel("Occupancy Rate (%)", fontsize=12, color='steelblue')
    ax_twin2.set_ylabel("Average Queue Length", fontsize=12, color='red')
    ax_twin.set_title("Hourly Usage Patterns", fontsize=12)
    ax_twin.grid(True, alpha=0.3)

    # Combine legends
    lines1, labels1 = ax_twin.get_legend_handles_labels()
    lines2, labels2 = ax_twin2.get_legend_handles_labels()
    ax_twin.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    plt.tight_layout()
    plt.savefig(f"enhanced_occupancy_analysis_{system_code}.png", dpi=300, bbox_inches='tight')
    plt.close()

# 3. Summary Statistics and Correlation Analysis
print("\n=== SUMMARY STATISTICS ===")
for system_code in sample_system_codes:
    lot_data = df[df["SystemCodeNumber"] == system_code]

    print(f"\nSystem Code: {system_code}")
    print(f"Average Occupancy Rate: {lot_data['OccupancyRate'].mean():.1f}%")
    print(f"Peak Occupancy Rate: {lot_data['OccupancyRate'].max():.1f}%")
    print(f"Average Queue Length: {lot_data['QueueLength'].mean():.1f}")
    print(f"Max Queue Length: {lot_data['QueueLength'].max()}")

    # Price statistics
    price_cols = ['Price_Model1', 'Price_Model2', 'Price_Model3']
    for col in price_cols:
        print(f"{col} - Mean: ${lot_data[col].mean():.2f}, Std: ${lot_data[col].std():.2f}")

# 4. Correlation Analysis
correlation_data = df[['Price_Model1', 'Price_Model2', 'Price_Model3',
                       'Occupancy', 'QueueLength', 'OccupancyRate']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title("Correlation Matrix: Prices, Occupancy, and Queue Length", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig("correlation_matrix.png", dpi=300, bbox_inches='tight')
plt.close()


Analyzing data for 3 system codes: ['BHMBCCMKT01' 'BHMBCCTHL01' 'BHMEURBRD01']
Date range: 2016-10-04 07:59:00 to 2016-12-19 16:30:00

=== SUMMARY STATISTICS ===

System Code: BHMBCCMKT01
Average Occupancy Rate: 28.1%
Peak Occupancy Rate: 99.3%
Average Queue Length: 3.6
Max Queue Length: 11
Price_Model1 - Mean: $19.24, Std: $5.31
Price_Model2 - Mean: $11.50, Std: $0.12
Price_Model3 - Mean: $11.50, Std: $0.15

System Code: BHMBCCTHL01
Average Occupancy Rate: 74.5%
Peak Occupancy Rate: 104.1%
Average Queue Length: 4.1
Max Queue Length: 12
Price_Model1 - Mean: $32.84, Std: $13.92
Price_Model2 - Mean: $11.54, Std: $0.13
Price_Model3 - Mean: $11.35, Std: $0.40

System Code: BHMEURBRD01
Average Occupancy Rate: 64.4%
Peak Occupancy Rate: 100.0%
Average Queue Length: 4.0
Max Queue Length: 11
Price_Model1 - Mean: $30.99, Std: $12.26
Price_Model2 - Mean: $11.53, Std: $0.12
Price_Model3 - Mean: $11.27, Std: $0.46
